In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('./cleaned_data.csv')

In [3]:
df.head()

,Unnamed: 0,date,price,address,bedrooms,bathrooms,parking,area,median_price,population,average_age,zipcode,state
0,0,29 04 24,480000,"1307/60 A'beckett Street, Melbourne",2,1,0,53.0,5501000,47279,0,3000,Victoria
1,1,27 03 24,815000,"1103/108 Flinders Street, Melbourne",2,2,1,93.0,5501000,47279,0,3000,Victoria
2,2,29 02 24,3350000,"3801/35 Spring Street, Melbourne",3,2,2,175.0,10721000000,47279,0,3000,Victoria
3,3,29 02 24,3350000,"3801/35 Spring Street, Melbourne",3,2,2,175.0,10721000000,47279,0,3000,Victoria
4,4,8 12 23,590000,"508/181 Exhibition Street, Melbourne",2,1,1,80.0,5501000,47279,0,3000,Victoria


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54851 entries, 0 to 54850
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    54851 non-null  int64  
 1   date          54851 non-null  object 
 2   price         54851 non-null  int64  
 3   address       54851 non-null  object 
 4   bedrooms      54851 non-null  int64  
 5   bathrooms     54851 non-null  int64  
 6   parking       54851 non-null  int64  
 7   area          54851 non-null  float64
 8   median_price  54851 non-null  int64  
 9   population    54851 non-null  int64  
 10  average_age   54851 non-null  int64  
 11  zipcode       54851 non-null  int64  
 12  state         54851 non-null  object 
dtypes: float64(1), int64(9), object(3)
memory usage: 5.4+ MB


In [5]:
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder

df['state'] = df['state'].str.strip()
df_encoded = df.copy()


le = LabelEncoder()
df_encoded['date'] = le.fit_transform(df_encoded['date'])
df_encoded['address'] = le.fit_transform(df_encoded['address'])
df_encoded['average_age'] = le.fit_transform(df_encoded['average_age'])

In [6]:
x = df_encoded.drop('state', axis=1)
y = df_encoded['state']

le_state = LabelEncoder()
y_encoded = le_state.fit_transform(y)

smote = SMOTE(random_state=42)
x_resampled, y_resampled = smote.fit_resample(x, y_encoded)

y_resampled_decoded = le_state.inverse_transform(y_resampled)

df = x_resampled
df['state'] = y_resampled_decoded

In [7]:
df['state'].value_counts()

Victoria              17092
New South Wales       17092
Queensland            17092
Western Australia     17092
Southern Australia    17092
Northern Territory    17092
Tasmania              17092
Name: state, dtype: int64

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119644 entries, 0 to 119643
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Unnamed: 0    119644 non-null  int64  
 1   date          119644 non-null  int64  
 2   price         119644 non-null  int64  
 3   address       119644 non-null  int64  
 4   bedrooms      119644 non-null  int64  
 5   bathrooms     119644 non-null  int64  
 6   parking       119644 non-null  int64  
 7   area          119644 non-null  float64
 8   median_price  119644 non-null  int64  
 9   population    119644 non-null  int64  
 10  average_age   119644 non-null  int64  
 11  zipcode       119644 non-null  int64  
 12  state         119644 non-null  object 
dtypes: float64(1), int64(11), object(1)
memory usage: 11.9+ MB


In [9]:
from sklearn.preprocessing import StandardScaler

df['state'] = le_state.fit_transform(df['state'])

X = df[['bedrooms', 'bathrooms', 'parking', 'area', 'state']]
y = df['price']
scaler = StandardScaler()
x = scaler.fit_transform(X)

In [10]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=2345)

In [11]:
from catboost import CatBoostRegressor

model = CatBoostRegressor()
model.fit(x_train, y_train)

Learning rate set to 0.083319
0:	learn: 593386.8996400	total: 53.5ms	remaining: 53.4s
1:	learn: 581145.7741891	total: 59.1ms	remaining: 29.5s
2:	learn: 570531.3567251	total: 64.3ms	remaining: 21.4s
3:	learn: 560303.0885977	total: 70.1ms	remaining: 17.5s
4:	learn: 551431.4506774	total: 75.7ms	remaining: 15.1s
5:	learn: 544098.0424354	total: 81.2ms	remaining: 13.5s
6:	learn: 537949.6301658	total: 86.5ms	remaining: 12.3s
7:	learn: 532073.7261762	total: 90.2ms	remaining: 11.2s
8:	learn: 527548.2386269	total: 94.3ms	remaining: 10.4s
9:	learn: 522869.5001375	total: 97.6ms	remaining: 9.66s
10:	learn: 518529.8084282	total: 100ms	remaining: 9.03s
11:	learn: 515098.5150966	total: 103ms	remaining: 8.47s
12:	learn: 511384.0074068	total: 105ms	remaining: 7.99s
13:	learn: 508352.9999221	total: 107ms	remaining: 7.57s
14:	learn: 505654.4718978	total: 110ms	remaining: 7.21s
15:	learn: 503239.9199923	total: 113ms	remaining: 6.92s
16:	learn: 500612.3446529	total: 115ms	remaining: 6.64s
17:	learn: 498200.

In [21]:
from sklearn.metrics import r2_score,root_mean_squared_error, mean_absolute_error, mean_squared_error

r2=r2_score(y_test, model.predict(x_test))
mse=mean_squared_error(y_test, model.predict(x_test))
mae=mean_absolute_error(y_test, model.predict(x_test))
rmse=root_mean_squared_error(y_test, model.predict(x_test))

print(f"r2 score: {r2}")
print(f"mse: {mse}")
print(f"mae: {mae}")
print(f"rmse: {rmse}")
# r2_score(y_test, model.predict(x_test))


r2 score: 0.4914885068193938
mse: 190929031851.12036
mae: 257046.93185816286
rmse: 436954.2674595596


In [13]:
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

model2 = XGBRegressor()
model2.fit(x_train, y_train)
r2_score(y_test, model2.predict(x_test))

0.48766753946111996

In [14]:
model3 = DecisionTreeRegressor()
model3.fit(x_train, y_train)
r2_score(y_test, model3.predict(x_test))

0.19920113185733945

In [15]:
model4 = RandomForestRegressor()
model4.fit(x_train, y_train)
r2_score(y_test, model4.predict(x_test))

0.41236191977151704

In [16]:
model5 = SVR(kernel="linear")
model5.fit(x_train, y_train)
r2_score(y_test, model5.predict(x_test))

-0.008841119805544473

In [17]:
model6 = KNeighborsRegressor()
model6.fit(x_train, y_train)
r2_score(y_test, model6.predict(x_test))

0.42660079497869485